In [1]:
import squigglepy as sq
from squigglepy.numbers import K, M, B
import numpy as np
import copy
import general_equations as ge

SIMULATIONS = 10*M

def mix_models(models, weights=None):
    if weights is None:
        weights = [1/len(models) for model in models]
    
    model_mix = sq.mixture(models, weights)
    sample = sq.sample(model_mix, n=SIMULATIONS)
    return sample

def dalys_per_1000_by_reducing_risk_mixture_model(models_dict, weights_dict, power):
    p_success_models = models_dict['p_success']
    risk_models = models_dict['risk']
    bp_per_billion_models = models_dict['bp_per_billion_if_success']
    times_credit_if_success_models = models_dict['times_credit_if_success']

    p_success_weights = weights_dict['p_success']
    risk_weights = weights_dict['risk']
    bp_per_billion_weights = weights_dict['bp_per_billion_if_success']
    times_credit_if_success_weights = weights_dict['times_credit_if_success']


    p_success = mix_models(p_success_models, p_success_weights)
    risk = mix_models(risk_models, risk_weights)
    bp_per_billion_if_success = mix_models(bp_per_billion_models, bp_per_billion_weights)
    times_credit_if_success = mix_models(times_credit_if_success_models, times_credit_if_success_weights)

    dalys_risk_weighted = ge.one_model_dalys_per_1000_by_reducing_risk(risk, power, p_success, bp_per_billion_if_success, times_credit_if_success)

    return dalys_risk_weighted

def find_percentile_of_value(value, array_of_values):
    prop_under =  sum(array_of_values < value)/len(array_of_values)
    return prop_under

def print_outputs(dalys_per_1000_unweighted, risk_function, interpretation_function, const):
    dalys_per_1000_ghd = ge.dalys_per_1000_certain_intervention()
    ev_dalys_per_1000 = np.mean(dalys_per_1000_unweighted)
    print("Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: {}".format(ev_dalys_per_1000))
    print("Standard deviation DALYs/$1000 for no risk-aversion: {}".format(np.std(dalys_per_1000_unweighted)))
    risk_weighted_dalys_per_1000 = risk_function(dalys_per_1000_unweighted, const)
    ghd_dalys_per_1000_risk_weighted = risk_function([dalys_per_1000_ghd], const)
    rw_ev_dalys_per_1000 = np.mean(risk_weighted_dalys_per_1000)
    print("Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = {}: {}".format(const, rw_ev_dalys_per_1000))

    print("GHD Risk-Weighted DALYs/$1000, shape = {}: {}".format(const, ghd_dalys_per_1000_risk_weighted))

    if rw_ev_dalys_per_1000 > ghd_dalys_per_1000_risk_weighted[0]: 
        print("X-risk reduction is better than GHD in terms of Risk-weighted expected value")
    else:
        print("GHD is better than X-risk reduction in terms of Risk-weighted expected value")

    p_needed = interpretation_function(const)
    return rw_ev_dalys_per_1000, p_needed

def get_percentiles_to_make_equal_to_ghd(dalys_per_1000_unweighted, risk_function, interpretation_function, c=0):
    dalys_per_1000_ghd = ge.dalys_per_1000_certain_intervention()
    print("DALYs per $1000 if you choose GHD: ", dalys_per_1000_ghd)
    prop_under = find_percentile_of_value(dalys_per_1000_ghd, dalys_per_1000_unweighted)
    print("If you choose X-risk, there's a {} chance of saving fewer lives than the certain GHD spending".format(ge.get_pct(prop_under)))

    rw_ev_dalys_per_1000, p_needed = print_outputs(dalys_per_1000_unweighted, risk_function, interpretation_function, c)
    return rw_ev_dalys_per_1000, p_needed
    

## Models

In [2]:
## Population Times the Present We Can Take Credit For if we Solve X-risk
low_pop_credit = sq.lognorm(10, 1000)
high_pop_credit = sq.lognorm(47.5, 40000)
pop_weights = [0.9, 0.1]
pop_models = [low_pop_credit, high_pop_credit]

## Probability of Success
p_success = sq.lognorm(0.4,0.8, lclip=0.01, rclip=0.99)
p_success_models = [p_success]
p_success_weights = [1]

## Risk
risk = sq.lognorm(0.1, 0.25, lclip=0.01, rclip=0.5)
risk_models = [risk]
risk_weights = [1]

## Basis Points Per Billion If Success
bp_model1 = sq.lognorm(0.001, 0.001)
bp_model2= sq.lognorm(0.001, 2.2)
bp_model1n = sq.uniform(-0.0005, -0.00005)
bp_model2n= sq.norm(-.5, -0.0005)

bp_models = [bp_model1, bp_model2, bp_model1n, bp_model2n]
bp_per_billion_weights = [0.9/2, 0.1/2, 0.95/2, 0.05/2]

models = {'p_success': p_success_models, 'risk': risk_models, 
        'bp_per_billion_if_success': bp_models, 'times_credit_if_success': pop_models}

weights = {'p_success': p_success_weights, 'risk': risk_weights, 
           'bp_per_billion_if_success': bp_per_billion_weights, 'times_credit_if_success': pop_weights}


## CRRA Utility Function

#### Equations

In [3]:

def crra_interpretation(const):
    if const == 1:
        p_needed = np.log(100)/np.log(10*K)
    else:
        p_needed = (100**(1-const)-1)/((10*K)**(1-const)-1)
    print("This level of risk aversion is like being indifferent to choosing to take $100 with certainty or a {} chance of getting $10K and {} chance of getting $1".format(ge.get_pct(p_needed), ge.get_pct(1-p_needed)))
    return p_needed

def crra_risk_aversion(dalys, c=0):
    dalys2 = copy.deepcopy(dalys)
    if 0 < c < 1:
        for idx, value in enumerate(dalys):
            if value > 0:
                dalys2[idx] = (value**(1-c)-1)/(1-c)
            else:
                dalys2[idx] = -(20**(1-c)-1)/(1-c)
    elif c == 1: 
        for idx, value in enumerate(dalys):
            if value > 0:
                dalys2[idx] = np.log(value)
            else:
                dalys2[idx]  = -np.log(20)

    elif c == 0:
        dalys2 = dalys
    return np.array(dalys2)

def multiple_crra_risk_aversion(models, weights):
    coefficients = [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.35, 0.38, 0.4, 0.42, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    dalys_per_1000_unweighted = dalys_per_1000_by_reducing_risk_mixture_model(models, weights, 1)
    for c in coefficients:
        print("-------------------------------------------------------------------")
        if c == 0:
            print("Coefficient of Relative Risk Aversion: c = {} (No Risk Aversion)".format(c))
        else:
            print("Coefficient of Relative Risk Aversion: c = {}".format(c))
        get_percentiles_to_make_equal_to_ghd(dalys_per_1000_unweighted, crra_risk_aversion, crra_interpretation, c)

#### CRRA Utility

In [4]:
multiple_crra_risk_aversion(models, weights)

100%|██████████| 10000000/10000000 [00:09<00:00, 1044916.52it/s]


-------------------------------------------------------------------
Coefficient of Relative Risk Aversion: c = 0 (No Risk Aversion)
DALYs per $1000 if you choose GHD:  20.0
If you choose X-risk, there's a 89.68% chance of saving fewer lives than the certain GHD spending
Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: 1212.7336295813434
Standard deviation DALYs/$1000 for no risk-aversion: 379079.22515179665
Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = 0: 1212.7336295813434
GHD Risk-Weighted DALYs/$1000, shape = 0: [20.]
X-risk reduction is better than GHD in terms of Risk-weighted expected value
This level of risk aversion is like being indifferent to choosing to take $100 with certainty or a 0.99% chance of getting $10K and 99.01% chance of getting $1
-------------------------------------------------------------------
Coefficient of Relative Risk Aversion: c = 0.01
DALYs per $1000 if you choose GHD:  20.0
If you choose X-risk, there's a 89.68% chance of

## Prospect Theory Method

#### Equations

In [10]:
def prospect_theory_utility(dalys, cs): 
    a, b, l = cs
    certain = ge.dalys_per_1000_certain_intervention()
    dalys2 = copy.deepcopy(dalys)
    for idx, value in enumerate(dalys):
        if value > certain:
            dalys2[idx] = (value - certain)**a
        else:
            dalys2[idx] = -l*(certain - value)**b

    return np.array(dalys2)

def prospect_theory_utility_estimates(models, weights, coefficients, loss_aversion):
    c_tuples = [(a, a, l) for a in coefficients for l in loss_aversion]
    ghd_best = []
    dalys_per_1000_unweighted = dalys_per_1000_by_reducing_risk_mixture_model(models, weights, 1)
    for cs in c_tuples:
        a, b, l = cs
        print("-------------------------------------------------------------------")
        print("Coefficients of Prospect Theory: a = {}, b = {}, l = {}".format(a, b, l))
        rw_ev_dalys_per_1000, p_needed = get_percentiles_to_make_equal_to_ghd(dalys_per_1000_unweighted, prospect_theory_utility, prospect_theory_indifference, cs)
        if rw_ev_dalys_per_1000 < 0:
            ghd_best.append((a, l, p_needed))

    return ghd_best

def prospect_theory_indifference(cs):
    a, b, l = cs
    p = (l*100**b)/(l*100**b+9900**a)
    print("This level of risk aversion and loss aversion is like being indifferent between choosing to take $100 with certainty versus {} chance of getting $10K versus nothing".format(ge.get_pct(p)))

    return p 

def pick_best_combo(ghd_best):
    if len(ghd_best) > 0:
        best = ghd_best[0]
        for combo in ghd_best:
            if combo[2] < best[2]:
                best = combo
        return best
    else:
        return None
    
def pick_worst_combo(ghd_best):
    if len(ghd_best) > 0:
        worst = ghd_best[0]
        for combo in ghd_best:
            if combo[2] > worst[2]:
                worst = combo
        return worst
    else:
        return None

def prospect_theory_all(models, weights, cs, loss_av):
    ghd_best = prospect_theory_utility_estimates(models, weights, cs, loss_av)
    print(ghd_best)
    best = pick_best_combo(ghd_best)
    if best is not None:
        print("The best combination is: a = {}, lambda = {}, p_needed = {}".format(best[0], best[1], best[2]))
    worst = pick_worst_combo(ghd_best)
    if worst is not None:
        print("The worst combination is: a = {}, lambda = {}, p_needed = {}".format(worst[0], worst[1], worst[2]))

coefficients = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.625, 0.65, 0.675, 0.7, 0.75, 0.78, 0.8, 0.82, 0.85, 0.88, 0.9, 1]

#### Loss Aversion Not Allowed

In [11]:
loss_aversion=[1]
prospect_theory_all(models, weights, coefficients, loss_aversion)

100%|██████████| 10000000/10000000 [00:29<00:00, 338784.71it/s]


-------------------------------------------------------------------
Coefficients of Prospect Theory: a = 0.01, b = 0.01, l = 1
DALYs per $1000 if you choose GHD:  20.0
If you choose X-risk, there's a 89.70% chance of saving fewer lives than the certain GHD spending
Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: 1148.6599624222843
Standard deviation DALYs/$1000 for no risk-aversion: 458858.1358790407
Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = (0.01, 0.01, 1): -0.8171090396959678
GHD Risk-Weighted DALYs/$1000, shape = (0.01, 0.01, 1): [-0.]
GHD is better than X-risk reduction in terms of Risk-weighted expected value
This level of risk aversion and loss aversion is like being indifferent between choosing to take $100 with certainty versus 48.85% chance of getting $10K versus nothing
-------------------------------------------------------------------
Coefficients of Prospect Theory: a = 0.05, b = 0.05, l = 1
DALYs per $1000 if you choose GHD:  20.0
If yo

#### Loss Aversion Allowed

In [13]:
loss_aversion=[2.25]
prospect_theory_all(models, weights, coefficients, loss_aversion)

100%|██████████| 10000000/10000000 [00:08<00:00, 1127212.58it/s]


-------------------------------------------------------------------
Coefficients of Prospect Theory: a = 0.01, b = 0.01, l = 2.25
DALYs per $1000 if you choose GHD:  20.0
If you choose X-risk, there's a 89.70% chance of saving fewer lives than the certain GHD spending
Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: 1196.0544001979083
Standard deviation DALYs/$1000 for no risk-aversion: 411429.0992061566
Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = (0.01, 0.01, 2.25): -1.9727816617514535
GHD Risk-Weighted DALYs/$1000, shape = (0.01, 0.01, 2.25): [-0.]
GHD is better than X-risk reduction in terms of Risk-weighted expected value
This level of risk aversion and loss aversion is like being indifferent between choosing to take $100 with certainty versus 68.24% chance of getting $10K versus nothing
-------------------------------------------------------------------
Coefficients of Prospect Theory: a = 0.05, b = 0.05, l = 2.25
DALYs per $1000 if you choose GHD: